**Задание 1**

НАЗВАНИЕ Мэтчинг данных

[Условие](https://docs.google.com/spreadsheets/d/1wrqhJfdlENbqpVxPcNksSzR2z9xblj9t8eRepg_`Bfn8/edit#gid=1805447557)

ОПИСАНИЕ
В листе 1 есть имена,клиентов их контактны, менеджеры,  которые с ними работают и другие данные.
В листе 2 обезличенные номера телефонов (новые заявки).
Необходимо определить, работали ли уже с клиентами из листа 2, кто менеджер клиента, какой статус у такого клиента, и, собственно имя.
Выяснить кол-во таких клиентов, а так же кол-во НОВЫХ клиентов,  с которыми не работал менеджер
Образ результата: таблица,  в которой четко понятно кто из новых клиентов с кем работает из менеджеров,  а кто новый клиент.


In [1]:
#Импорт библиотек
import pandas as pd
import numpy as np
import re

Иморт и корректировка данных

In [2]:
df1 = pd.read_csv("https://docs.google.com/spreadsheets/d/1wrqhJfdlENbqpVxPcNksSzR2z9xblj9t8eRepg_Bfn8/export?gid=1805447557&format=csv", header=0)
df1 = df1.drop(["Unnamed: 7", "Unnamed: 8", "Unnamed: 9"], axis=1)
df1.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Менеджер        875 non-null    object
 1   ФИО             875 non-null    object
 2   Телефон         871 non-null    object
 3   Город           874 non-null    object
 4   Уровень дохода  875 non-null    object
 5   Готов Оплатить  863 non-null    object
 6   Статус          874 non-null    object
dtypes: object(7)
memory usage: 48.0+ KB


,Менеджер,ФИО,Телефон,Город,Уровень дохода,Готов Оплатить,Статус
0,Бадма-Церенов Джин,Байгозина Жанара,77011188868,Astana,1 000 000 - 5 000 000,"Готов(-а). Но есть вопросы, которые хочу задать",НДЗ
1,Бадма-Церенов Джин,Макаров Игорь,79099370081,Москва,1 000 000 - 5 000 000,"Готов(-а). Но есть вопросы, которые хочу задать",Отказ
2,Бадма-Церенов Джин,Ильнур,79225177733,Samui,1 000 000 - 5 000 000,"Да, я готов(-а) менять свою жизнь!",НДЗ
3,Бадма-Церенов Джин,Почапский Дмитрий Александрович,79381177333,Ростов-на-Дону,1 000 000 - 5 000 000,"Готов(-а). Но есть вопросы, которые хочу задать",Отказ
4,Бадма-Церенов Джин,Богомолов Павел Александрович,79877769058,Оренбург,1 000 000 - 5 000 000,"Готов(-а). Но есть вопросы, которые хочу задать",Оплатил


In [3]:
#Проверка строчек без телефона
df1.query('Телефон.isnull()')

,Менеджер,ФИО,Телефон,Город,Уровень дохода,Готов Оплатить,Статус
8,Бадма-Церенов Джин,Ерлан,NaN,Алматы,100 000 - 300 000,"Очень хочу, но денег нет",НДЗ
56,Бадма-Церенов Джин,Локтионов Владимир Владимирович,NaN,"Dubai, Магнитогорск",100 000 - 300 000,"Да, я готов(-а) менять свою жизнь!",Отказ
106,Бадма-Церенов Джин,Кострова Инна Владимировна,NaN,Санкт-Петербург,100 000 - 300 000,"Очень хочу, но денег нет",Отказ
128,Володин Андрей,79082662047,NaN,Пермь,300 000 - 1 000 000,"Готов(-а). Но есть вопросы, которые хочу задать",Отказ


In [4]:
#В ФИО у этого человека его телефон, а поле телефона пустое
df1.loc[128, 'Телефон'] = df1.loc[128, 'ФИО']

In [5]:
#Если в телефоне указано что-то помимо телефона, то всё стирается.
#Если цифр нет, то ничего не меняется
for i in df1["Телефон"].astype('string').str.extract(r'([^0-9])').dropna().index:
  if re.search(r'\d+', df1.loc[i, 'Телефон']):
    df1.loc[i, 'Телефон'] = re.search(r'\d+', df1.loc[i, 'Телефон']).group()
  else:
    df1.loc[i, 'Телефон'] = np.NaN

In [6]:
df2 = pd.read_csv("https://docs.google.com/spreadsheets/d/1wrqhJfdlENbqpVxPcNksSzR2z9xblj9t8eRepg_Bfn8/export?gid=1682825470&format=csv", header=0, dtype=object)
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Тел. Клиента  499 non-null    object
dtypes: object(1)
memory usage: 4.0+ KB


,Тел. Клиента
0,93737721
1,576193913
2,991750594
3,3725843628
4,7700640000


Сопостовляем номера из двух таблиц.

In [7]:
result = df2.merge(df1, how="left", left_on="Тел. Клиента", right_on="Телефон")[["Тел. Клиента", "Менеджер"]]
result.info()
result

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499 entries, 0 to 498
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Тел. Клиента  499 non-null    object
 1   Менеджер      326 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


,Тел. Клиента,Менеджер
0,93737721,Гельвер Сабина
1,576193913,Горбачева Алена
2,991750594,Калачева Алина
3,3725843628,NaN
4,7700640000,Горбачева Алена
...,...,...
494,79171141382,Кузнецова Даша
495,79171654450,Бадма-Церенов Джин
496,79172442888,Бадма-Церенов Джин
497,79172578734,Логинова Ксюша


499 номеров, у 326 уже есть менеджер => 429 - 326 = 173 новых клиентов.

Номера в РФ могут начинаться +7 (7) либо 8. Возможно какие-то одинаковые номера записаны разными способами в разных таблицах.

In [8]:
result.query('Менеджер.isnull()')["Тел. Клиента"].astype('string').str.extract(r'(^8+[0-9]{10}$)').dropna()

,0


В таблице с новыми номерами, которым ещё не нашли пару, нет номеров начинающихся на 8.

In [9]:
df1["Телефон"].astype('string').str.extract(r'(^8+[0-9]{10}$)').dropna()

,0
115,89024560577
185,89095772536
410,89219867623
447,89516522154
492,80168429881
516,89378779596
546,80626534464
640,89834415173
641,89036056888
762,87025303229


Есть номера начинающиеся на 8 таблице старых номеров.

In [10]:
new_clients_with_7_10numbers_phone = result.query('Менеджер.isnull()')["Тел. Клиента"].astype('string').str.extract(r'(^7+([0-9]{10})$)').dropna().reset_index()
old_clients_with_8_10numbers_phone = df1["Телефон"].astype('string').str.extract(r'(^8+([0-9]{10})$)').dropna().join(df1['Менеджер'], how='left')
more_results = new_clients_with_7_10numbers_phone.merge(old_clients_with_8_10numbers_phone, how='inner', left_on=1, right_on=1).set_index('index')
more_results

,0_x,1,0_y,Менеджер
index,,,,
129,79024560577,9024560577,89024560577,Бадма-Церенов Джин
164,79036056888,9036056888,89036056888,Калачева Алина
261,79093061555,9093061555,89093061555,Логинова Ксюша


In [11]:
for i in more_results.index:
  result.loc[i, "Менеджер"] = more_results.loc[i, "Менеджер"]

In [12]:
result.query('Менеджер.isnull()')

,Тел. Клиента,Менеджер
3,3725843628,NaN
5,7922147687,NaN
6,7968795155,NaN
31,37066726064,NaN
41,72111188861,NaN
...,...,...
483,79168839938,NaN
484,79168882289,NaN
490,79169868620,NaN
493,79171089884,NaN


In [13]:
result

,Тел. Клиента,Менеджер
0,93737721,Гельвер Сабина
1,576193913,Горбачева Алена
2,991750594,Калачева Алина
3,3725843628,NaN
4,7700640000,Горбачева Алена
...,...,...
494,79171141382,Кузнецова Даша
495,79171654450,Бадма-Церенов Джин
496,79172442888,Бадма-Церенов Джин
497,79172578734,Логинова Ксюша


Новых клиентов на самом деле 170 (на 3 меньше).

В таблице result клиенты с менеджерами, те у кого нет менеджера (NaN) -- новые клиенты.